In [1]:
import numpy as np
from scipy.linalg import block_diag, null_space
from lie_groups import su2, so31
from lie_groups.util import comm

In [2]:
comm(so31.Jp(1), so31.Jp(2)) - so31.Jp(3)

array([[0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j]])

In [3]:
comm(so31.Jp(1), so31.Jm(2))

array([[0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j]])

In [4]:
def proj(X, e):
    return (e.conj() * X).sum() / (e.conj() * e).sum()

In [5]:
def vector_rep(X):
    """(1/2, 1/2) rep of so(3, 1)"""
    def proj(X, e):
        return (e.conj() * X).sum() / (e.conj() * e).sum()

    basis_in = []
    basis_out = []
    for i in range(1, 4):
        basis_in.append(so31.Jp(i))
        basis_out.append(np.kron(su2.S(i), np.eye(2)))
        basis_in.append(so31.Jm(i))
        basis_out.append(np.kron(np.eye(2), su2.S(i)))

    coefs = [proj(X, e) for e in basis_in]
    return np.sum([c * e for c, e in zip(coefs, basis_out)], axis=0)


In [7]:
vector_rep(so31.J1)

array([[0.+0.j , 0.-0.5j, 0.-0.5j, 0.+0.j ],
       [0.-0.5j, 0.+0.j , 0.+0.j , 0.-0.5j],
       [0.-0.5j, 0.+0.j , 0.+0.j , 0.-0.5j],
       [0.+0.j , 0.-0.5j, 0.-0.5j, 0.+0.j ]])

In [8]:
vector_rep(so31.J3)

array([[0.-1.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+1.j]])

In [35]:
def solve_equivalence(elts1, elts2):
    """Solves for an equivalence of representations.
    
    Specifically compute the set of matrices S such that SA = BS
    for each (A, B) in zip(elts1, elts2). Any such invertible matrix
    gives an isomorphism of the representations.
    """
    dim = len(elts1[0])
    blocks = []
    for a, b in zip(elts1, elts2):
        blocks.append(np.kron(np.eye(dim), a.T) - np.kron(b, np.eye(dim)))
    return null_space(np.concatenate(blocks, axis=0), rcond=1e-10).reshape(-1, dim, dim)

In [39]:
elts = [so31.J1, so31.J2, so31.K1]
rep = [vector_rep(e) for e in elts]
S = solve_equivalence(elts, rep).round(4)[0]
S /= S.ravel()[np.abs(S).argmax()]
display(S)

array([[-0.+0.j,  1.+0.j,  0.-1.j,  0.+0.j],
       [ 1.+0.j, -0.+0.j, -0.+0.j, -1.+0.j],
       [-1.+0.j,  0.-0.j, -0.+0.j, -1.+0.j],
       [ 0.-0.j, -1.+0.j,  0.-1.j,  0.+0.j]])

In [29]:
S @ so31.K2 - vector_rep(so31.K2) @ S

array([[0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j]])

In [42]:
elts = [so31.J1, so31.K2, so31.K1]
rep = [vector_rep(e) for e in elts]
S_alt = solve_equivalence(elts, rep).round(4)[0]
S_alt /= S_alt.ravel()[np.abs(S_alt).argmax()]
S_alt

array([[ 0.-0.j,  1.+0.j,  0.-1.j,  0.+0.j],
       [ 1.+0.j, -0.+0.j,  0.+0.j, -1.+0.j],
       [-1.+0.j, -0.+0.j,  0.+0.j, -1.+0.j],
       [ 0.+0.j, -1.+0.j,  0.-1.j, -0.+0.j]])